<a href="https://colab.research.google.com/github/ritiktaneja/PS1/blob/master/Classify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install earthengine-api

In [0]:
!earthengine authenticate 

Instructions for updating:
non-resource variables are not supported in the long term
Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=JpY6BdoaWc7beeSuSm5HppxOjtZcMUhps7kZtUUq0bU&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/0gEz0_MUTDypmahzthZnmhh7v-OeBQzON15T6w1L_cGdi5QBWqrW_jU

Successfully saved authorization token.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
import ee
ee.Initialize()

In [0]:
loc = ee.Geometry.Polygon([[[91.91778932373563,25.85213724917264], [93.26361451904813,25.85213724917264], [93.26361451904813,26.743483718578567],  [91.91778932373563,26.743483718578567], [91.91778932373563,25.85213724917264]]])

In [0]:
im_col=ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(loc).filterDate(ee.Date('2017-01-01'),ee.Date('2017-12-31')).filterMetadata('CLOUD_COVER','less_than',2).sort('CLOUD_COVER',True)
num_im=im_col.size()
print(num_im.getInfo())

9


In [0]:
im=im_col.mosaic()


In [0]:
import folium
#EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

In [0]:
# Mapdisplay: Display GEE objects using folium.
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

In [0]:
from IPython.display import Image
Image(url=im.getThumbUrl({'min': 0, 'max': 3000, 'dimensions': 512, 'bands':['B4','B3','B2']}))

In [0]:
ClpIm=im.clip(loc)

In [0]:
from IPython.display import Image
Image(url=ClpIm.getThumbUrl({'min': 0, 'max': 3000, 'dimensions': 512, 'bands':['B4','B3','B2']}))

In [0]:
def addmNDWI(image):
  NIR=image.select('B5')
  RED=image.select('B4')
  mNDWI=image.expression('((B1-B2)/(B1+B2))',{'B1':NIR,'B2':RED}).rename('mNDWI')
  image=image.addBands(mNDWI)
  return image

In [0]:
print('Bandnames of the original image are',ClpIm.bandNames().getInfo())
mNDWI_im=addmNDWI(ClpIm)
print('Bandnames of the image with added mNDWI are',mNDWI_im.bandNames().getInfo())


Bandnames of the original image are ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11', 'sr_aerosol', 'pixel_qa', 'radsat_qa']
Bandnames of the image with added mNDWI are ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11', 'sr_aerosol', 'pixel_qa', 'radsat_qa', 'mNDWI']


In [0]:
mNDWI=mNDWI_im.select('mNDWI')
#mask_im=mNDWI.lte(0.22)
ndvi_palette='306466, 3460ff, 9cab68, cccc66, 40b922, 9c8448, 6e462c'
Image(url=mNDWI.getThumbUrl({'min': 0.3, 'max': 0.9, 'dimensions': 512, 'palette': ndvi_palette}))

In [0]:
forest = [
       ee.Feature(ee.Geometry.Polygon([[92.94450504538713,26.31771538397151], [92.94609291312395,26.315561203946526],[92.94699413535295,26.31367626356549],[92.95476181266008,26.31425328938818],[92.96008331534563,26.31694603856107],[92.9592679238051,26.325639344328096],[92.95244438407121,26.323331450666284],[92.94450504538713,26.31771538397151]]),{'landuse':0}),
       ee.Feature(ee.Geometry.Polygon([[93.18888406013889,26.40622916130637],[93.18896989082737,26.39654245344181],[93.19772462105198,26.39700374367961],[93.20261697029514,26.39869512544728],[93.20261697029514,26.40592165926661],[93.20090035652561,26.406690412829537],[93.18888406013889,26.40622916130637]]),{'landuse':0}),
       ee.Feature(ee.Geometry.Polygon([[93.2074587579948,26.163992786346153],[93.20883204901043,26.158446055030456],[93.21011950933757,26.15351540577169],[93.21398189031902,26.153900620248905],[93.21818759405437,26.154825129805275],[93.21758677923503,26.16460907354298],[93.2074587579948,26.163992786346153]]),{'landuse':0}),
       ee.Feature(ee.Geometry.Polygon([[92.23942901007524,26.007161335981618],[92.25286151282182,26.00800986640841],[92.25668097845903,26.010863969210977],[92.2540631424605,26.010979674618156],[92.23942901007524,26.007161335981618]]),{'landuse':0}),
       ee.Feature(ee.Geometry.Polygon([[92.22724105231157,26.009822615415644],[92.22960139624468,26.009012667189943],[92.2338929306685,26.010054028168287],[92.2347941528975,26.01387227274035],[92.24097396246782,26.017034765131726],[92.24178935400835,26.019310164270234],[92.23930026404253,26.020968478226493],[92.23595286719194,26.019657255176618],[92.22724105231157,26.009822615415644]]),{'landuse':0})
]
ftr = ee.FeatureCollection(forest)
plantation = [
              ee.Feature(ee.Geometry.Polygon([[92.94122274886217,26.457514780661167],[92.94199522505846,26.45816792908594],[92.94298227797594,26.45847529176837],[92.94444139968004,26.458206349466106],[92.94437702666369,26.456631103365094],[92.94740255843249,26.456976890940854],[92.94712360869494,26.457898986065654],[92.94526899317339,26.45887531488696],[92.94625604609087,26.46058499888971],[92.94250095347002,26.45983581464312],[92.94093454340532,26.458894524960616],[92.93908918360307,26.458337431523127],[92.93930376032426,26.457204026208377],[92.93966854075029,26.45718481585265],[92.94122274886217,26.457514780661167]]),{'landuse':1}),
              ee.Feature(ee.Geometry.Polygon([[92.97616804102495,26.46356247782638],[92.97616804102495,26.464388474464947],[92.97567451456621,26.465348928218813],[92.97524536112383,26.46600203219327],[92.97445142725542,26.46615570317804],[92.97389352778032,26.465368137212142],[92.97427976587846,26.46467661143268],[92.97492349604204,26.463735361334262],[92.97571742991045,26.463408803379156],[92.97616804102495,26.46356247782638]]),{'landuse':1}),
              ee.Feature(ee.Geometry.Polygon([[92.87403654682862,26.44093573380818],[92.87588190663087,26.440974159937337],[92.87725519764649,26.440974159937337],[92.8790576421045,26.440993372997124],[92.88030218708741,26.4414160595015],[92.8794009648584,26.442645684151394],[92.87858557331788,26.443241278873174],[92.87781309712159,26.442376704880648],[92.87590336430299,26.442491981787853],[92.87410091984498,26.442299853545066],[92.87238430607545,26.442011660580164],[92.87264179814088,26.441588976260885],[92.87403654682862,26.44093573380818]]),{'landuse':1}),
              ee.Feature(ee.Geometry.Polygon([[92.65278310517283,26.532643903319126],[92.65707463959666,26.533335020239686],[92.66256780365916,26.537097694894126],[92.66196698883982,26.541704883557298],[92.6565596554658,26.544469107938728],[92.65346975068064,26.54554406609627],[92.65089483002635,26.541935238133416],[92.65037984589549,26.53387255274465],[92.6510664914033,26.53302785767775],[92.65278310517283,26.532643903319126]]),{'landuse':1}),
              ee.Feature(ee.Geometry.Polygon([[92.63209790924998,26.525348526431245],[92.6345869992158,26.524964546381522],[92.63819188813181,26.52473415773518],[92.63673276642771,26.527038023390737],[92.63965100983592,26.529034669554697],[92.63596029023142,26.52795955670546],[92.63209790924998,26.525348526431245]]),{'landuse':1}),
              ee.Feature(ee.Geometry.Polygon([[92.75091605729176,26.540126291298446],[92.7524180943401,26.5383985934581],[92.75344806260182,26.538859315426286],[92.75422053879811,26.54039504195728],[92.75422053879811,26.54250663236825],[92.75224643296315,26.541777178257274],[92.75091605729176,26.540126291298446]]),{'landuse':1}),
              ee.Feature(ee.Geometry.Polygon([[93.16707938147806,25.96168254719751],[93.16840975714945,25.961065191519566],[93.16909640265726,25.96044783260286],[93.16999762488626,25.96118094595589],[93.16905348731302,25.962299899636605],[93.16780894233011,25.962299899636605],[93.16737978888773,25.962184146300984],[93.16707938147806,25.96168254719751]]),{'landuse':1})
              
]
ptr = ee.FeatureCollection(plantation)
water = [
         ee.Feature(ee.Geometry.Polygon([[92.8973609950491,26.626681866034993],[92.89813347124539,26.615632504541416],[92.9094631221243,26.613944313602204],[92.9237110164114,26.61578597520903],[92.91976280474148,26.62207809520924],[92.90731735491238,26.626681866034993],[92.8973609950491,26.626681866034993]]),{'landuse':2}),
         ee.Feature(ee.Geometry.Polygon([[92.67012221564937,26.58901244979617],[92.6685772632568,26.587554128903477],[92.66823394050289,26.585097967513438],[92.67089469184566,26.586326054795258],[92.67793280830074,26.587093602656697],[92.6828251575439,26.588168161017343],[92.68771750678707,26.58839842221095],[92.70050627937007,26.5874006203604],[92.70454032172847,26.588321668531197],[92.70454032172847,26.591468527200277],[92.70067794074703,26.59154527876997],[92.69793135871578,26.592773296885696],[92.69355399360347,26.592696546139482],[92.68874747504879,26.593080299355936],[92.68479926337886,26.593464051285686],[92.68085105170894,26.595843284529952],[92.67716033210445,26.59983414534512],[92.67304045905757,26.6052062381693],[92.67115218391109,26.60443881178187],[92.67278296699214,26.6001411288733],[92.67484290351558,26.597838732336022],[92.67707450141597,26.594615299354547],[92.67647368659664,26.592466293591993],[92.67201049079586,26.592773296885696],[92.6678047870605,26.594538549843563],[92.66316992988277,26.59522929358933],[92.66316992988277,26.593847801928717],[92.66909224738765,26.58970322689265],[92.67012221564937,26.58901244979617]]),{'landuse':2}),
         ee.Feature(ee.Geometry.Polygon([[93.19580215479914,26.65403231065542],[93.19786209132258,26.647741950386475],[93.20850509669367,26.652804950502652],[93.20747512843195,26.6554130750529],[93.19580215479914,26.65403231065542]]),{'landuse':2}),
         ee.Feature(ee.Geometry.Polygon([[91.91909416192698,26.275047000857324],[91.91917999261545,26.26627293257144],[91.92467315667795,26.270044323891575],[91.93076713555979,26.273507738602678],[91.93857772821116,26.27566270004303],[91.9475899505012,26.277971542897035],[91.94286926263499,26.282666048364533],[91.9285355376594,26.277432817006503],[91.92184074395823,26.27458522432448],[91.91909416192698,26.275047000857324]]),{'landuse':2})
         
         
]
wtr = ee.FeatureCollection(water)
sand = [
        ee.Feature(ee.Geometry.Polygon([[93.2017917919934,26.63746411755139],[93.20273592956664,26.637540838253702],[93.20608332641723,26.63685035007786],[93.20788577087524,26.638691642604954],[93.21123316772582,26.641376807668802],[93.21226313598754,26.642987876405815],[93.21054652221801,26.64436877437722],[93.21346476562621,26.646440090025184],[93.21106150634887,26.649201785775666],[93.20488169677856,26.6482812279476],[93.20076182373168,26.642911159362257],[93.20050433166625,26.637540838253702],[93.2017917919934,26.63746411755139]]),{'landuse':3}),
        ee.Feature(ee.Geometry.Polygon([[93.12797739990356,26.641913833106333],[93.13020899780395,26.637540838253702],[93.13364222534301,26.639382119650527],[93.13252642639281,26.64214398609225],[93.13973620422485,26.647283948591493],[93.1373329449475,26.648588081381654],[93.1355305004895,26.6462099456975],[93.13106730468871,26.645135932701674],[93.131839780885,26.643601610898997],[93.12883570678832,26.64214398609225],[93.12797739990356,26.641913833106333]]),{'landuse':3}),
        ee.Feature(ee.Geometry.Polygon([[92.81371311118244,26.578259320981974],[92.8142280953133,26.56812649328199],[92.81474307944416,26.562829431203337],[92.8197212593758,26.564748684881202],[92.82512859274982,26.570659784235104],[92.82839015891193,26.578259320981974],[92.83173755576252,26.583862696807273],[92.83542827536701,26.590233325233527],[92.8266735451424,26.59146135742501],[92.82032207419513,26.584092966660407],[92.8211803810799,26.579026922918644],[92.82049373557209,26.57641705534316],[92.81706050803302,26.57987127910655],[92.81448558737873,26.57941072195763],[92.81371311118244,26.578259320981974]]),{'landuse':3}),
        ee.Feature(ee.Geometry.Polygon([[91.96415527337717,26.281665595912376],[91.96578605645823,26.277278894864118],[91.97085006707835,26.279510745945732],[91.96638687127756,26.28220430215092],[91.96415527337717,26.281665595912376]]),{'landuse':3}),
        ee.Feature(ee.Geometry.Polygon([[91.92518814080881,26.26565718357889],[91.9270764159553,26.26527233880024],[91.93025215142893,26.266657774033455],[91.93222625726389,26.26442567579707],[91.9366036223762,26.26619596412598],[91.93711860650706,26.26935162854854],[91.94106681817698,26.27119880694433],[91.94286926263499,26.27304595594548],[91.93789108270335,26.27212238511931],[91.9325695800178,26.27089094591961],[91.92802055352854,26.26873589588473],[91.92424400323557,26.267042614219818],[91.92518814080881,26.26565718357889]]),{'landuse':3})

]
stra = ee.FeatureCollection(sand)
agriculture = [
               ee.Feature(ee.Geometry.Polygon([[93.06983483373176,25.971212884533628],[93.07172310887825,25.963342133349993],[93.07605755864631,25.962261774025155],[93.07991993962776,25.962956291872683],[93.07622922002327,25.964769735814777],[93.08094990788948,25.969168185854624],[93.08369648992073,25.971367349204904],[93.07949078618537,25.971058697179952],[93.07794582555823,25.96997842045826],[93.07562840520393,25.970055572505366],[93.0718947702552,25.971907488299717],[93.06983483373176,25.971212884533628]]),{'landuse':4}),
               ee.Feature(ee.Geometry.Polygon([[93.00810895703239,25.95573701261543],[93.00943933270378,25.953421795168495],[93.01179967663688,25.951145120275104],[93.00926767132682,25.950450532728826],[93.01600538037223,25.952418520133214],[93.01458917401237,25.955505492919922],[93.0108126237194,25.956277223467275],[93.00931058667106,25.95616146420773],[93.00810895703239,25.95573701261543]]),{'landuse':4}),
               ee.Feature(ee.Geometry.Polygon([[92.8142617057722,25.965653344011287],[92.81542042006663,25.961910701810698],[92.81825283278636,25.961061838564348],[92.82305935134104,25.960328724469274],[92.82537677992991,25.964225752267723],[92.82087066878489,25.968238398490058],[92.81748035659007,25.96827698096249], [92.8142617057722,25.965653344011287]]),{'landuse':4}),
               ee.Feature(ee.Geometry.Polygon([[92.72824691915002,26.013718516445596],[92.73004936360803,26.00600475299519],[92.73665832662073,26.007470407043222],[92.74026321553674,26.012175804295218],[92.73648666524377,26.01587827939972],[92.72824691915002,26.013718516445596]]),{'landuse':4}),
               ee.Feature(ee.Geometry.Polygon([[92.63677902483425,26.680455509073973],[92.6417572047659,26.679228433157245],[92.64347381853543,26.674780172262874],[92.64416046404324,26.670331737859815],[92.64776535295925,26.670331737859815],[92.6475936915823,26.6757005163393],[92.6501686122366,26.677081018527474],[92.64707870745144,26.68014874133284],[92.64261551165066,26.682449479272677],[92.63677902483425,26.684596792793535],[92.63609237932644,26.68290962128919],[92.63677902483425,26.680455509073973]]),{'landuse':4}),
               ee.Feature(ee.Geometry.Polygon([[92.57429428362332,26.660513889429176],[92.58116073870144,26.660974119946637],[92.58064575457058,26.66419568159546],[92.57772751116238,26.663735464071507],[92.57429428362332,26.660513889429176]]),{'landuse':4})
               
]
atr = ee.FeatureCollection(agriculture)

    


In [0]:
TrainFC=ftr.merge(ptr.merge(wtr.merge(stra.merge(atr))))

In [0]:
bands=ClpIm.bandNames()
training = ClpIm.select(bands).sampleRegions(
  collection=TrainFC,
  properties=['landuse'],
  scale=30)

In [0]:
trainedMD = ee.Classifier.minimumDistance().train(training, 'landuse', bands)
trainedSVM = ee.Classifier.libsvm().train(training, 'landuse', bands)
trainedRF = ee.Classifier.randomForest().train(training, 'landuse', bands)

In [0]:
ClassifiedImageMD=ClpIm.select(bands).classify(trainedMD)
ClassifiedImageSVM=ClpIm.select(bands).classify(trainedSVM)
ClassifiedImageRF=ClpIm.select(bands).classify(trainedRF)

In [0]:

Image(url=ClassifiedImageMD.getThumbUrl({'min': 0, 'max': 4, 'dimensions': 900, 'palette': ['37a100','49df0a','0ce9ff','f1ff18','91e35e']}))

In [0]:
Image(url=ClassifiedImageRF.getThumbUrl({'min': 0, 'max': 4, 'dimensions': 900, 'palette': ['37a100','49df0a','0ce9ff','f1ff18','91e35e']}))

In [0]:
Image(url=ClassifiedImageSVM.getThumbUrl({'min': 0, 'max': 4, 'dimensions': 900, 'palette': ['37a100','49df0a','0ce9ff','f1ff18','91e35e']}))

In [0]:
import folium

# Define a function for displaying Earth Engine image tiles to folium map. 
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer
# Create a folium map object where we will load our images and maps onto.
my_map = folium.Map(location=[26.2786,92.5538], zoom_start=10, height=500)
# Add the image to the map object in the style defined by the add_ee_layer function. The visualization parameters in curly braces can be replaced by a variable for readability
my_map.add_ee_layer(ClassifiedImageMD, {'min': 0,'max': 4,'palette': ['37a100','49df0a','0ce9ff','f1ff18','91e35e']}, 'Classified Image MD')
my_map.add_ee_layer(ClassifiedImageRF, {'min': 0,'max': 4,'palette': ['37a100','49df0a','0ce9ff','f1ff18','91e35e']}, 'Classified Image RF')
my_map.add_ee_layer(ClassifiedImageSVM, {'min': 0,'max': 4,'palette': ['37a100','49df0a','0ce9ff','f1ff18','91e35e']}, 'Classified Image SVM')

# Adding layer controls
my_map.add_child(folium.LayerControl())
# Click on the map to get a pop up of latitude and longitude
folium.LatLngPopup().add_to(my_map)
# To display the map, use the display function and pass the map object as its argument
display(my_map)






 

In [0]:
compare = ClassifiedImageSVM.sampleRegions(TrainFC,['landuse'],30)
errormatrix = compare.errorMatrix('landuse','classification')
print(errormatrix.accuracy().getInfo())
print(errormatrix.kappa().getInfo())
print(errormatrix.getInfo())

1
1
[[5839, 0, 0, 0, 0], [0, 2283, 0, 0, 0], [0, 0, 9664, 0, 0], [0, 0, 0, 6887, 0], [0, 0, 0, 0, 4485]]
